In [2]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.utils import ImageReader
from IPython.display import FileLink, display
from datetime import datetime
import os, math, io
import qrcode

def _fit(text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    s = max_font
    while s > min_font and pdfmetrics.stringWidth(t, font_name, s) > box_w:
        s -= 0.5
    return max(min_font, s)

def _val(prefill, dotted):
    if "." not in dotted:
        return "" if prefill.get(dotted) is None else str(prefill.get(dotted))
    cur = prefill
    for p in dotted.split("."):
        if not isinstance(cur, dict): return ""
        cur = cur.get(p, "")
    return "" if cur is None else str(cur)

def _fmt_date(x):
    if isinstance(x, datetime): return x.strftime("%Y-%m-%d")
    return "" if x is None else str(x)

def _normalize_dates(prefill):
    p = dict(prefill or {})
    if "created" in p: p["created"] = _fmt_date(p["created"])
    if "date" in p: p["date"] = _fmt_date(p["date"])
    return p

def _qr_reader(value, box_size=5, border=1):
    q = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_Q, box_size=box_size, border=border)
    q.add_data("" if value is None else str(value))
    q.make(fit=True)
    pil_img = q.make_image(fill_color="black", back_color="white").convert("RGB")
    buf = io.BytesIO()
    pil_img.save(buf, format="PNG")
    buf.seek(0)
    return ImageReader(buf)

class FormRenderer:
    def __init__(self, title, filename, page_size=A4, margins_mm=12, pad_mm=3,
                 label_font=("Helvetica","Helvetica-Bold"),
                 label_sizes=(10,7.4), value_font_max=11.5, title_sizes=(18,10)):
        self.title = title
        self.filename = filename
        self.page_size = page_size
        self.w, self.h = page_size
        self.m = margins_mm*mm
        self.pad = pad_mm*mm
        self.left, self.right = self.m, self.w - self.m
        self.usable_w = self.right - self.left
        self.label_font = label_font
        self.label_sizes = label_sizes
        self.value_font_max = value_font_max
        self.title_sizes = title_sizes
        self.c = canvas.Canvas(filename, pagesize=page_size)
        self.c.setTitle(title)
        self.cursor = None

    def draw_title(self, text=None, suffix=None, top_offset_mm=5):
        t = self.title if text is None else text
        if suffix: t = f"{t} {suffix}"
        fs = _fit(t, self.label_font[1], self.title_sizes[0], self.title_sizes[1], self.usable_w)
        self.c.setFont(self.label_font[1], fs)
        self.c.setFillColor(colors.black)
        y = self.h - self.m - top_offset_mm*mm
        self.c.drawCentredString(self.w/2, y, t)
        self.cursor = y - 7*mm
        return fs

    def draw_qr_below_title(self, value, size_mm=26, label=None, gap_mm=4):
        if value in (None, ""):
            return
        img_reader = _qr_reader(value)
        side = size_mm * mm
        x = (self.w - side) / 2.0
        y_top = self.cursor
        self.c.drawImage(img_reader, x, y_top - side, width=side, height=side, mask='auto')
        if label is not None:
            self.c.setFont(self.label_font[0], 8.5)
            self.c.drawCentredString(self.w/2, y_top - side - 3.5*mm, str(label))
            self.cursor = y_top - side - (gap_mm+6)*mm
        else:
            self.cursor = y_top - side - gap_mm*mm

    def _new_page(self, title_suffix=None):
        self.c.showPage()
        self.c.setTitle(self.title)
        self.draw_title(suffix=title_suffix)

    def draw_field_block(self, rows, prefill, row_h_mm=12.0, gap_mm=2.6, heights=None):
        heights = heights or {"single": row_h_mm*mm, "multi": (row_h_mm+4)*mm}
        gap = gap_mm*mm
        for cols, hspec in rows:
            h = heights.get(hspec, heights["single"] if len(cols)>1 else heights["multi"])
            if self.cursor - h < self.m + 24:
                self._new_page("(cont.)")
            self._draw_row(self.cursor, h, cols, prefill)
            self.cursor -= h + gap

    def _draw_row(self, top_y, height, cols, prefill):
        c = self.c
        c.setLineWidth(1.0)
        c.setStrokeColor(colors.black)
        c.rect(self.left, top_y-height, self.usable_w, height, stroke=1, fill=0)
        acc = 0.0
        x = self.left
        for i, (field, ratio) in enumerate(cols):
            cw = self.usable_w * ratio
            acc += ratio
            if i < len(cols)-1:
                x_next = self.left + self.usable_w * acc
                c.line(x_next, top_y-height, x_next, top_y)
            label = field.get("label","")
            key = field.get("key","")
            fs = _fit(label, self.label_font[0], self.label_sizes[0], self.label_sizes[1], cw - 2*self.pad)
            c.setFont(self.label_font[0], fs)
            c.setFillColor(colors.black)
            c.drawString(x + self.pad, top_y - self.pad - fs*0.15, label)
            v = _val(prefill, key)
            if v:
                vfs = _fit(v, self.label_font[1], self.value_font_max, 8, cw - 2*self.pad)
                c.setFont(self.label_font[1], vfs)
                baseline = top_y - (height*0.62)
                c.drawString(x + self.pad, baseline, v)
            x += cw

    def label(self, text, down_mm=0):
        if down_mm: self.cursor -= down_mm*mm
        self.c.setFont(self.label_font[1], 10.5)
        self.c.drawString(self.left, self.cursor, text)
        self.cursor -= 6.8*mm

    def draw_list_section(self, items, columns, row_h_mm=8.6, header_bg=0.95, cell_pad_mm=1.4, gap_after_mm=8):
        if items is None: items = []
        if not isinstance(items, list): items = list(items)
        weights = [max(0.01, col.get("weight",1)) for col in columns]
        total_wt = sum(weights)
        col_ws = [self.usable_w * (w/total_wt) for w in weights]
        row_h = row_h_mm*mm
        head_h = row_h
        need_h_min = head_h + row_h
        if self.cursor - need_h_min < self.m + 24:
            self._new_page("(cont.)")
        self._draw_list_header(col_ws, row_h, columns, header_bg, cell_pad_mm)
        body_top = self.cursor - head_h
        idx = 0
        while idx < max(1, len(items)):
            max_body_h = (body_top - self.m) - 10
            max_rows = max(1, int(math.floor(max_body_h/row_h)))
            end_idx = min(len(items), idx + max_rows)
            self._draw_list_body(col_ws, row_h, items, idx, end_idx, cell_pad_mm, columns)
            self.cursor = body_top - (end_idx-idx)*row_h - gap_after_mm*mm
            idx = end_idx
            if idx < len(items):
                self._new_page("(cont.)")
                self._draw_list_header(col_ws, row_h, columns, header_bg, cell_pad_mm)
                body_top = self.cursor - head_h

    def _draw_list_header(self, col_ws, row_h, columns, header_bg, cell_pad_mm):
        c = self.c
        pad = cell_pad_mm*mm
        total_w = sum(col_ws)
        c.setFillColorRGB(header_bg, header_bg, header_bg)
        c.rect(self.left, self.cursor - row_h, total_w, row_h, fill=1, stroke=0)
        c.setFillColor(colors.black)
        c.setLineWidth(0.9)
        c.setStrokeColor(colors.black)
        c.rect(self.left, self.cursor - row_h, total_w, row_h, stroke=1, fill=0)
        cx = self.left
        for cw in col_ws[:-1]:
            cx += cw
            c.line(cx, self.cursor - row_h, cx, self.cursor)
        self.c.setFont(self.label_font[1], 8.8)
        x = self.left
        for i, col in enumerate(columns):
            hdr = col.get("label", col.get("key",""))
            self.c.drawString(x + pad, self.cursor - row_h + 2.6*mm, str(hdr))
            x += col_ws[i]

    def _draw_list_body(self, col_ws, row_h, items, start_idx, end_idx, cell_pad_mm, columns):
        c = self.c
        pad = cell_pad_mm*mm
        total_w = sum(col_ws)
        body_top = self.cursor - row_h
        c.setLineWidth(0.8)
        c.setStrokeColor(colors.black)
        c.rect(self.left, body_top - (end_idx - start_idx)*row_h, total_w, (end_idx - start_idx)*row_h, stroke=1, fill=0)
        cx = self.left
        for cw in col_ws[:-1]:
            cx += cw
            c.line(cx, body_top - (end_idx - start_idx)*row_h, cx, body_top)
        for r in range(1, (end_idx - start_idx)):
            y = body_top - r*row_h
            c.line(self.left, y, self.left + total_w, y)
        c.setFont("Helvetica", 9.0)
        for r, idx in enumerate(range(start_idx, end_idx)):
            row = items[idx] if idx < len(items) else {}
            row_top = body_top - r*row_h
            baseline = row_top - (row_h*0.68)
            x = self.left
            for i, colw in enumerate(col_ws):
                cbox_w = colw - 2*mm
                key = columns[i]["key"]
                sval = "" if row.get(key) is None else str(row.get(key))
                fs = _fit(sval, "Helvetica", 9.2, 6.6, cbox_w)
                c.setFont("Helvetica", fs)
                c.drawString(x + 1.2*mm, baseline, sval)
                x += colw

    def save(self):
        self.c.save()
        try: display(FileLink(self.filename))
        except: pass
        return os.path.abspath(self.filename)

def render_po_form_prefilled(cfg=None):
    cfg = cfg or {}
    title = cfg.get("title","PURCHASE ORDER – FUEL")
    filename = cfg.get("filename","purchase_order_fuel_item_pumpname_fixed_a4.pdf")
    page_size = cfg.get("page_size", A4)
    margins_mm = cfg.get("margins_mm", 12)
    pad_mm = cfg.get("pad_mm", 3)
    label_font = cfg.get("label_font", ("Helvetica","Helvetica-Bold"))
    label_font_sizes = cfg.get("label_font_sizes",(10,7.4))
    value_font_max = cfg.get("value_font_max", 11.5)
    title_font_sizes = cfg.get("title_font_sizes",(18,10))
    prefill = _normalize_dates(cfg.get("prefill",{}))
    if "type" not in prefill: prefill["type"] = "fuel"
    if "form_name" not in prefill: prefill["form_name"] = "Purchase Order – Fuel"
    fuel_items = list(prefill.get("fuel") or [])
    r = FormRenderer(title, filename, page_size, margins_mm, pad_mm,
                     label_font, label_font_sizes, value_font_max, title_font_sizes)
    r.draw_title()
    qr_value = prefill.get("current_form_qr_code","")
    r.draw_qr_below_title(qr_value, size_mm=26, label=qr_value, gap_mm=4)
    fields_top = [
        ([({"key":"form_name","label":"Form Name"},0.34),
          ({"key":"id","label":"PO ID"},0.26),
          ({"key":"created","label":"Created On"},0.20),
          ({"key":"date","label":"PO Date"},0.20)],"single"),
        ([({"key":"tin","label":"TIN"},0.40),
          ({"key":"location","label":"Location"},0.60)],"single"),
        ([({"key":"type","label":"Type"},0.25),
          ({"key":"previous_form_qr_code","label":"Previous Form QR"},0.75)],"single"),
    ]
    r.draw_field_block(fields_top, prefill, row_h_mm=cfg.get("row_h_mm",12.0), gap_mm=cfg.get("gap_mm",2.6))
    r.label("FUEL PO LINE ITEMS", down_mm=0)
    columns = cfg.get("columns") or [
        {"key":"pump_name","label":"Pump Name","weight":22},
        {"key":"po_number","label":"PO No","weight":18},
        {"key":"plate_number","label":"Plate No","weight":16},
        {"key":"route","label":"Route","weight":32},
        {"key":"driver","label":"Driver","weight":24},
        {"key":"product","label":"Product","weight":18},
        {"key":"quantity","label":"Qty","weight":12},
    ]
    r.draw_list_section(
        fuel_items if fuel_items else [{}],
        columns,
        row_h_mm=cfg.get("table_row_h_mm",8),
        header_bg=0.95,
        cell_pad_mm=1.6,
        gap_after_mm=8
    )
    r.label("RESPONSIBLE PERSONNEL", down_mm=2)
    fields_bottom = [
        ([({"key":"cashier_employee_number","label":"Cashier Emp No."},0.50),
          ({"key":"recorder_employee_number","label":"Recorder Emp No."},0.50)],"single"),
    ]
    r.draw_field_block(fields_bottom, prefill, row_h_mm=cfg.get("row_h_mm_bottom",12.0), gap_mm=cfg.get("gap_mm_bottom",2.6))
    return r.save()

cfg = {
    "filename":"purchase_order_fuel_item_pumpname_fixed_a4.pdf",
    "prefill":{
        "id":"PO-20251005-001",
        "current_form_qr_code":"POF-20251005-001",
        "previous_form_qr_code":"POF-20250930-004",
        "type":"fuel",
        "tin":"123-456-789-000",
        "location":"Main Depot – Tagbilaran",
        "created":"2025-10-05",
        "date":"2025-10-06",
        "fuel":[
            {"pump_name":"Pump 1 – Diesel","po_number":"2025100601","plate_number":"AAA-1234","route":"Depot → Branch 1","driver":"Juan Dela Cruz","product":"Diesel","quantity":"80"},
            {"pump_name":"Pump 2 – Regular","po_number":"2025100602","plate_number":"BBB-5678","route":"Depot → Branch 2","driver":"Maria Santos","product":"Regular","quantity":"60"},
            *[{"pump_name":"","po_number":"","plate_number":"","route":"","driver":"","product":"","quantity":""} for _ in range(10)]
        ],
        "cashier_employee_number":"",
        "recorder_employee_number":""
    }
}
render_po_form_prefilled(cfg)


h:\github4\jefstore-gasstations-backend\reportlab\purchase_orders\filled\purchase_order_fuel_item_pumpname_fixed_a4.pdf

'h:\\github4\\jefstore-gasstations-backend\\reportlab\\purchase_orders\\filled\\purchase_order_fuel_item_pumpname_fixed_a4.pdf'